Wave packet tunneling
============

This notebook does some simulations of a moving wave packet approaching a 
finite-size barrier. 

The numerical technique used here is taken from 

    P.B.Visscher, "A fast explicit algorithm for the time-dependent Schrödinger equation",
    Computers in Physics 5, 596 (1991),
    https://doi.org/10.1063/1.168415
    
This method is simpler to code than e.g. Runge-Kutta, but preserves the
wave function norm and is stable for relatively large time steps.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
def norm(x, psi):
    """Compute the norm of the wave function."""
    dx  = x[1]-x[0]
    nrm = np.dot( psi, np.conj(psi) )*dx
    return nrm

def gaussian(x, x0, width, p):
    """Put a gaussian wave function on the lattice. The
    `x0`, `width` and `p` variables are in physical 
    units."""
    dx  = x[1]-x[0]
    pu  = np.exp( -(x-x0)**2/width**2 + 1j*x*p)
    nrm = norm(x, pu)
    pu /= np.sqrt(nrm)
    return pu

Evolution of the wave function is done by splitting it in real and imaginary
parts, and then updating them on a staggered lattice. For details see the
Visscher paper mentioned at the top of this notebook. The `evolve` function 
below takes care of this.

In [ ]:
def evolve(H, dt, R, I):
    """Given R(t) and I(t+(1/2)dt), produce R(t+dt) and I(t+(3/2)dt)."""
    #  R(t+dt)     = R(t)        + dt H I(t+1/2dt)
    #  I(t+3/2 dt) = I(t+1/2 dt) - dt H R(t+dt)
    nR = R + dt*np.dot(H, I)
    nI = I - dt*np.dot(H, nR)
    return nR, nI

Finally, we need some simple function to generate a step potential:

In [ ]:
def Vstep(x, iv, V0):
    """Generate a step potential in the range given by `iv`,
    which is in physical units."""
    dx     = x[1]-x[0]
    V      = np.zeros(len(x))
    loc    = np.where( (x > iv[0]) & (x < iv[1]) )
    V[loc] = V0
    return V

Some basic parameters to set the lattice size and range, and create
an initial wave packet with some momentum:

In [ ]:
N  = 800     # number of lattice points
Bl = -25.0   # left boundary in physical units
Br = 25.0    # right boundary in physical units
p  = 5       # momentum in physical units

dx = (Br-Bl)/(N-1)
x  = np.linspace(Bl, Br, N)
psi0 = gaussian(x, -20, width=2, p=p)

In [ ]:
%%capture
figsize=(16,10)
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(16,10))   

In [ ]:
%%capture
def init():
    """Initialise the evolution parameters and plots for a new simulation."""
    global psi0, H, line, R, I, dt, V
    plt.clf()
    R = np.real(psi0)
    I = np.imag(psi0)  # should be dt away
    line=[0,0,0,0,0,0]
    nR = R.copy()
    nI = I.copy()
    H = (2*np.identity(N) - np.diag( np.ones(N-1), 1) - np.diag( np.ones(N-1), -1))/2/dx**2 + np.diag(V)
    maxdt = 1/(1/dx**2+V0)
    dt=0.4*maxdt #0.005
    print("Stable? dt < 1/(1/dx**2 + max(abs(V))): ", dt < maxdt)
    YL=0.6
    axs[0].set_ylim(-YL, YL)
    axs[1].set_ylim(-YL, YL)
    axs[2].set_ylim(0,0.4)
    line[0]=axs[0].plot(x, R, 'b')[0]
    line[1]=axs[1].plot(x, I, 'r')[0]
    line[2]=axs[2].plot(x, x, 'k')[0]
    line[3]=axs[3].plot(x, V, 'k')[0]
    line[4]=plt.text(0,1.2, '', fontsize=15, transform=axs[0].transAxes)
    line[5]=plt.text(0.3,1.2, '', fontsize=15, transform=axs[0].transAxes)    
    plt.text(-0.1,0.5, '$Re(\psi)$', fontsize=15, transform=axs[0].transAxes)
    plt.text(-0.1,0.5, '$Im(\psi)$', fontsize=15, transform=axs[1].transAxes)
    plt.text(-0.1,0.5, '$|\psi|^2$', fontsize=15, transform=axs[2].transAxes)
    plt.text(-0.1,0.5, '$V(x)$', fontsize=15, transform=axs[3].transAxes)    
    return line

In [ ]:
def update_plot(n):
    """Evolve the wave function for 100 steps and then update the plots."""
    global R, I, nR, nI, line
    for i in range(100):
        nR, nI = evolve(H, dt, R, I)
        R, I = nR, nI
    line[0].set_data(x, nR)
    line[1].set_data(x, nI)
    psisq = R*R + nI*I
    nrmsq=np.sum(psisq)*dx
    line[2].set_data(x, psisq)
    R, I = nR, nI
    line[4].set_text('$|\psi|^2 = {}$'.format(nrmsq))
    line[5].set_text('$frame = {}$'.format(n))    
    return line

First a simulation without potential, just to see how the wave function spreads.

In [ ]:
%%capture
V0=10
V=np.zeros(len(x))
init()
anim = animation.FuncAnimation(fig, update_plot, frames=30, interval=80, repeat=False, blit=False )

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
%%capture
V0=10
V = Vstep(x, [0, 1.0], V0)
anim = animation.FuncAnimation(fig, update_plot, init_func=init, frames=30, interval=80, repeat=False, blit=False)

In [ ]:
HTML(anim.to_jshtml())

Questions? kasper.peeters@durham.ac.uk